In [1]:
import torch 
from torch import nn 
from torch.utils.data import DataLoader 


In [3]:
from torchvision import datasets
from torchvision.transforms import ToTensor


In [4]:
# Domain specific libraries 
# Torchtext, torchvision, torchaudio etc 
# all of them include datasets

In [5]:
# Download training data from the open datasets 
training_data = datasets.FashionMNIST(
    root = "data", 
    train = True, 
    download = True, 
    transform=ToTensor() 
)

In [6]:
# Download the test data from the open datasets 
test_data = datasets.FashionMNIST(
    root = "data", 
    train = False, 
    download = True, 
    transform=ToTensor() 
)

In [7]:
# Pass Dataset as an argument to the DataLoader 
# This wraps the iterable over dataset 
# supports automatic batching, sampling, shuffling and multiprocess data loading 

In [8]:
# Here we define a batch size of 64 
# each element in the data loader iterable will return a batch of 64 features and labels 

In [9]:
batch_size = 64 

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size) 
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# iterate over the test data loader 
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}") 
    print(f"Shape of y: {y.shape}, dtype: {y.dtype}") 
    break 

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]), dtype: torch.int64


In [10]:
# LOADING DATA IN PYTORCH ?? READ MORE 

In [11]:
# CREATE MODELS 
# DEFINE A NUERAL NETWORK 
# CREATE A CLASS THAT INHERITS FROM NN.MODULE 
# DEFINE THE LAYERS IN THE __INIT__ METHOD 
# SPECIFY HOW DATA WILL PASS THROUGH THE NETWORK IN THE FORWARD FUNCTION 


In [12]:
device = "cuda" if torch.cuda.is_available() else "mps"  if torch.backends.mps.is_available() else "cpu" 
print("Using {} device".format(device))

Using mps device


In [13]:
# DEFINE THE MODEL 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten() 
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), 
            nn.ReLU(), 
            nn.Linear(512, 512), 
            nn.ReLU(), 
            nn.Linear(512, 10), 
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x) 
        logits = self.linear_relu_stack(x) 
        return logits
    
model = NeuralNetwork().to(device) 
print(model) 

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [14]:
# OPTIMISE THE MODELS PARAMETERS 
# Loss function and an Optimiser 
loss_fn = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) 

In [15]:
# Single training loop 
# model makes a prediction on the training dataset(fed into it in batches) 
# backpropagates the prediction error to adjust the models parameters 

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) 
    model.train() 
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device) 

        # Compute prediction error 
        pred = model(X) 
        loss = loss_fn(pred, y) 

        # Backpropagation 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X) 
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [16]:
# Check models performance against the test dataset to ensure it is learning 

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset) 
    num_batches = len(dataloader) 
    model.eval() 
    test_loss, correct = 0, 0 
    with torch.no_grad(): 
        for X, y in dataloader: 
            X, y = X.to(device), y.to(device) 
            pred = model(X) 
            test_loss += loss_fn(pred, y).item() 
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() 
    test_loss /= num_batches 
    correct /= size 
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [17]:
# training is conducted over several iterations(epochs) 
# during each epoch, the model learns the parameters to make better predictions 
# print the models accuracy and loss at each epoch 
# like to see the accuracy increase and the loss decrease with each epoch 

epochs = 5 
for t in range(epochs): 
    print(f"Epoch {t+1}\n-------------------------------") 
    train(train_dataloader, model, loss_fn, optimizer) 
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302808  [    0/60000]
loss: 2.291238  [ 6400/60000]
loss: 2.277214  [12800/60000]
loss: 2.282506  [19200/60000]
loss: 2.254111  [25600/60000]
loss: 2.246540  [32000/60000]
loss: 2.264250  [38400/60000]
loss: 2.232526  [44800/60000]
loss: 2.225059  [51200/60000]
loss: 2.224681  [57600/60000]
Test Error: 
 Accuracy: 48.1%, Avg loss: 2.207457 

Epoch 2
-------------------------------
loss: 2.221290  [    0/60000]
loss: 2.191541  [ 6400/60000]
loss: 2.158401  [12800/60000]
loss: 2.188344  [19200/60000]
loss: 2.112699  [25600/60000]
loss: 2.117199  [32000/60000]
loss: 2.170691  [38400/60000]
loss: 2.103949  [44800/60000]
loss: 2.102556  [51200/60000]
loss: 2.112341  [57600/60000]
Test Error: 
 Accuracy: 51.4%, Avg loss: 2.064423 

Epoch 3
-------------------------------
loss: 2.100987  [    0/60000]
loss: 2.034875  [ 6400/60000]
loss: 1.971646  [12800/60000]
loss: 2.038448  [19200/60000]
loss: 1.895095  [25600/60000]
loss: 1.931775  [32000/600

In [18]:
# SAVE THE MODEL 
torch.save(model.state_dict(), "model.pth") 
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [19]:
# LOAD THE MODEL 
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [20]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [21]:
# What version of Python do you have?
import sys
import platform
import torch
import pandas as pd
import sklearn as sk

has_gpu = torch.cuda.is_available()
has_mps = getattr(torch,'has_mps',False)
device = "mps" if getattr(torch,'has_mps',False) \
    else "gpu" if torch.cuda.is_available() else "cpu"

print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if has_gpu else "NOT AVAILABLE")
print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AVAILABLE")
print(f"Target device is {device}")

Python Platform: macOS-13.1-arm64-arm-64bit
PyTorch Version: 2.0.0.dev20230217

Python 3.9.16 | packaged by conda-forge | (main, Feb  1 2023, 21:38:11) 
[Clang 14.0.6 ]
Pandas 1.4.2
Scikit-Learn 1.2.1
GPU is NOT AVAILABLE
MPS (Apple Metal) is AVAILABLE
Target device is mps


In [22]:
# PART 2 
import torch 
import numpy as np 

In [24]:
# INITIALISE A TENSOR 
# directly from the data 
data = [[1, 2], [3, 4]] 
x_data = torch.tensor(data) 
print(x_data) 

tensor([[1, 2],
        [3, 4]])


In [25]:
# from a numpy array 
np_array = np.array(data) 
x_np = torch.from_numpy(np_array) 
print(x_np) 

tensor([[1, 2],
        [3, 4]])


In [28]:
# from another tensor 

x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.2530, 0.3145],
        [0.3380, 0.1096]]) 



In [29]:
# from random or constant values 

shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.9942, 0.5537, 0.9043],
        [0.7085, 0.5480, 0.7504]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [30]:
# attributes of a tensor 
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


In [38]:
# standard numpy like indexing and slicing 
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

First row: tensor([1., 1., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [39]:
# Joining tensors 
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])


In [40]:
# Arithmetic operations 
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
# ``tensor.T`` returns the transpose of a tensor
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)


# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

In [41]:
# Single element tensors 
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

12.0 <class 'float'>


In [42]:
# Bride with numpy 
# tensor to numpy array 
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


In [43]:
# change in the tensor reflects in the numpy array 
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


In [44]:
# Numpy array to tensor 
n = np.ones(5)
t = torch.from_numpy(n)

In [45]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]
